## Libs

In [1]:
! pip3 install stringcase --user
! pip3 install matplotlib
! pip3 install nltk
! pip3 install wordcloud

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [4]:
import numpy as np
import pandas as pd
import psycopg2
import json

import uuid
import datetime
import re
import stringcase
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation
from wordcloud import WordCloud

[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


## Database

In [6]:
from modules.database_connector import Database

database = Database()
conn = database.get_connection()

Connected to database: [ec2-54-84-35-171.compute-1.amazonaws.com] @ 0 seconds


## Functions

In [2]:
char_colors = ['#FF7CB9','#979FEF']
csfont = {'fontname':'Helvetica'}

remove_special_char = lambda word: re.sub(r'[!.,\-";:~><^%$#@&*\(\)-+=_\'\?]','', word)

def calculate_age(birth): 
    age = today.year - birth.year - ((today.month, today.day) < (birth.month, birth.day)) 
    return age

def define_class(x):
    if x > 0.5:
        return 1
    elif x < 0.5:
        return 0
    else:
        return 0.5

def get_vocabulary(df):
    count_vectorizer = CountVectorizer(lowercase=False,stop_words=[])
    cv_fit = count_vectorizer.fit_transform(df['content'])
    word_list = count_vectorizer.get_feature_names()
    frequecy_array = cv_fit.toarray()
    count_list = frequecy_array.sum(axis=0)
    vocabulary = (dict(zip(word_list, count_list)))
    return vocabulary, frequecy_array, word_list

def normalize_dict(df):
    aux_df = dict({})
    for key in df.keys():
        aux_df[key] = np.round(df[key]/sum(df.values()),2)*100
    return aux_df

---

## Dataset Info

In [17]:
db_info = dict({})

### Users

In [9]:
try:
#     raise Exception
    users_df = pd.read_csv('data/users-df.csv')
except:
    print('Retrieving from original database...')
    users_df = pd.read_sql_query('select username, email, birth, gender from users;',conn)
    users_df['age'] = users_df['birth'].apply(calculate_age)
    users_df.to_csv('data/users-df.csv', index=False)

Retrieving from original database...


In [29]:
count_users = users_df.groupby(['gender'])['username'].count()
db_info['users'] = {}
db_info['users']['total'] = count_users['fem'] + count_users['masc'] + count_users['other']
db_info['users']['fem'] = count_users['fem']
db_info['users']['mas'] = count_users['masc']
db_info['users']['other'] = count_users['other']

In [18]:
temp_users = dict({})

temp_users['total'] = count_votes_by_gender['fem']+count_votes_by_gender['masc']
temp_users['female'] = count_votes_by_gender['fem']
temp_users['male'] = count_votes_by_gender['masc']
db_info['users'] = temp_users

In [30]:
db_info

{'users': {'total': 573, 'fem': 340, 'mas': 231, 'other': 2}}

### Comments

In [ ]:
from modules.classification.read_dataframe import Dataframe

data = Dataframe

In [3]:
try:
#     raise Exception
    labeled_comments = pd.read_csv('./data/labeled-comments.csv')
except:
    print('Retrieving from original database...')
    labeled_comments = pd.read_sql_query('select * from results;',conn)
    labeled_comments['label'] = labeled_comments['avg'].apply(lambda x: 1 if x > 0.5 else 0 if x < 0.5 else -1)
    labeled_comments['char-qty'] = labeled_comments['content'].apply(lambda comment: len(comment))
    labeled_comments['word-qty'] = labeled_comments['content'].apply(lambda comment: len(comment.lower().split(' ')))
    labeled_comments.to_csv('./data/labeled-comments.csv', index=False)

sexist_comments = labeled_comments[labeled_comments['avg'] > 0.5]
not_sexist_comments = labeled_comments[labeled_comments['avg'] < 0.5]
undefined_comments = labeled_comments[labeled_comments['avg'] == 0.5]

Retrieving from original database...


NameError: name 'pd' is not defined

#### Vocabulary

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vocabulary, freq, word_list = get_vocabulary(labeled_comments)

sexist_vocabulary, sexist_frequency_array, sexist_word_list = get_vocabulary(sexist_comments)
not_sexist_vocabulary, not_sexist_frequency_array, not_sexist_word_list = get_vocabulary(not_sexist_comments)
undefined_vocabulary, undefined_frequency_array, undefined_word_list = get_vocabulary(undefined_comments)
vectorizer = CountVectorizer()

print('Total terms:',len(vocabulary))

In [ ]:
pt_stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords_in_comments = [x for x in vocabulary.keys() if x in pt_stopwords]
print('Total stopwords:',len(stopwords_in_comments))

In [ ]:
number_in_voc = []
for x in vocabulary.keys():
    for char in x:
        if char in '0123456789' and x not in number_in_voc:
            try:
                int(x)
            except:
                number_in_voc.append(x)

print('Total alphanumeric terms:', len(number_in_voc))

In [ ]:
for term in vocabulary.keys():
    if term in number_in_voc:
        print(term, vocabulary[term])

In [ ]:
!pip install wordcloud -q
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
remove_words = ['MULHER', 'MULHERES', 'FEMINISMO', 'FEMINISTA', 'ASSÉDIO', 'FEMINISTAS', 'mulheres', 'mulher', 'assédio']
all_summary = " ".join(s.upper() for s in sexist_comments['content'])
all_summary = ' '.join([a for a in all_summary.split(' ') if a not in remove_words ])
wordcloud = WordCloud(stopwords=pt_stopwords + ['pra', 'desse', 'vc', 'nesse', 'algum', 'dessa', 'aqui', 'ser', 'cada'],
                      background_color='white', width=1600,
                      max_words=100,
                      height=800).generate(all_summary.lower())
fig, ax = plt.subplots(figsize=(16,8))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()
plt.imshow(wordcloud)
wordcloud.to_file('rafael.png',);

In [ ]:
all_summary

In [ ]:
temp_comments = dict({})
count_comments = labeled_comments.groupby(['label'])['comment_id'].count()
temp_comments['total'] = labeled_comments.shape[0]
temp_comments['sexist'] = {
    'total_qty': count_comments[1],
    'likes_qty': sexist_comments.sum()['likes'],
    'dislikes_qty': sexist_comments.sum()['dislikes'],
    'likes_qty_median': sexist_comments.median()['likes'],
    'dislikes_qty_median': sexist_comments.median()['dislikes'],
    'char_qty_median': sexist_comments.median()['char-qty'],
    'word_qty_median': sexist_comments.median()['word-qty']
}
temp_comments['not_sexist'] = {
    'total_qty': count_comments[0],
    'likes_qty': not_sexist_comments.sum()['likes'],
    'dislikes_qty': not_sexist_comments.sum()['dislikes'],
    'likes_qty_median': not_sexist_comments.median()['likes'],
    'dislikes_qty_median': not_sexist_comments.median()['dislikes'],
    'char_qty_median': not_sexist_comments.median()['char-qty'],
    'word_qty_median': not_sexist_comments.median()['word-qty']
}
temp_comments['undefined'] = {
    'total_qty': count_comments[-1],
    'likes_qty': undefined_comments.sum()['likes'],
    'dislikes_qty': undefined_comments.sum()['dislikes'],
    'likes_qty_median': undefined_comments.median()['likes'],
    'dislikes_qty_median': undefined_comments.median()['dislikes'],
    'char_qty_median': undefined_comments.median()['char-qty'],
    'word_qty_median': undefined_comments.median()['word-qty']
}
db_info['comments'] = temp_comments

#### Shared words between sexist and not sexist comments

In [ ]:
list_sexist_sorted_terms = []
for key, value in sorted(sexist_vocabulary.items(), key=lambda item: item[1]):
    list_sexist_sorted_terms.append(key)
list_sexist_sorted_terms.reverse()

shared_words = []

for word in list_sexist_sorted_terms:
    if word in not_sexist_vocabulary.keys():
        shared_words.append(word)

word_freq = {
    'word': [],
    'sexist-freq': [],
    'not-sexist-freq': [],
    'undefined-freq' : [],
    'diff' : []
}

for word in shared_words:
    word_freq['word'].append(word)
    if word in sexist_vocabulary.keys():
      word_freq['sexist-freq'].append(sexist_vocabulary[word])
    else:
      word_freq['sexist-freq'].append(0)
    if word in sexist_vocabulary.keys():
      word_freq['not-sexist-freq'].append(not_sexist_vocabulary[word])
    else:
      word_freq['not-sexist-freq'].append(0)
    if word in undefined_vocabulary.keys():
        word_freq['undefined-freq'].append(undefined_vocabulary[word])
    else:
        word_freq['undefined-freq'].append(0)
    word_freq['diff'] = sexist_vocabulary[word] - not_sexist_vocabulary[word]

word_freq = pd.DataFrame(word_freq)

# normalizind frequencies
sum_sexist = sum(word_freq['sexist-freq'])
word_freq['sexist-freq'] = word_freq['sexist-freq'].apply(lambda x: x/sum_sexist)
sum_not_sexist = sum(word_freq['not-sexist-freq'])
word_freq['not-sexist-freq'] = word_freq['not-sexist-freq'].apply(lambda x: x/sum_not_sexist)
undefined_sexist = sum(word_freq['undefined-freq'])
word_freq['undefined-freq'] = word_freq['undefined-freq'].apply(lambda x: x/undefined_sexist)
word_freq['diff'] = word_freq['sexist-freq'] - word_freq['not-sexist-freq']

sexist_words = word_freq[word_freq['diff'] > 0]
not_sexist_words = word_freq[word_freq['diff'] < 0]

In [ ]:
not_sexist_words

In [ ]:
relevant_sexist_words = list(sexist_words.sort_values(by=['diff'], ascending=False)['word'])

# 20 most relevant words to sexist comments
top20_sexist_words = sexist_words.sort_values(by ='diff',ascending=False ).head(20)
top20_sexist_words['perc_S'] = top20_sexist_words['sexist-freq'].apply(lambda x : x)
top20_sexist_words

In [ ]:
# print table to latex
for i, r in top20_sexist_words.iterrows():
    print("{%s} & {%0.6f} & {%0.6f} & {%0.6f}\\\\" %
          (r['word'], r['sexist-freq'], r['not-sexist-freq'], r['diff']))

### Histograms

In [ ]:
hiss = word_freq[word_freq['diff']>0].sort_values(by=['diff'], ascending=False)
hisn = word_freq[word_freq['diff']<0].sort_values(by=['diff'], ascending=True)
HS = list(hiss['sexist-freq'])
HN = list(hisn['not-sexist-freq'])

remove_special_char = lambda word: re.sub(r'[!.,\-";:~><^%$#@&*\(\)-+=_\'\?]','', word)
words = list(word_freq['word'])

duno_comments = [c for c in list(undefined_comments['content'])]
duno_matriz = []
words = list(hiss['word'])
for comment in duno_comments:
    hist = []
    for word in words:
        f = comment.split(' ').count(word)
        hist.append(f)
#     hist = [x/sum(hist) if sum(hist) > 0 else 0 for x in hist]
    duno_matriz.append(hist)
from scipy.spatial import distance
JSD_S = []
for line in duno_matriz:
    dist = distance.jensenshannon(HS, line)
    JSD_S.append(dist)

duno_matriz = []
words = list(hisn['word'])
for comment in duno_comments:
    hist = []
    for word in words:
        f = comment.split(' ').count(word)
        hist.append(f)
#     hist = [x/sum(hist) if sum(hist) > 0 else 0 for x in hist]
    duno_matriz.append(hist)
from scipy.spatial import distance
JSD_N = []
for line in duno_matriz:
    dist = distance.jensenshannon(HN, line)
    JSD_N.append(dist)

In [ ]:
db_info

In [ ]:
a= pd.DataFrame(JSD_S)
a = a.fillna(0)
group_values_S = {
    '<0.6': len(a[a[0]<0.6]),
    '0.6 < x < 0.7': len(a[(a[0]>=0.6) & (a[0]<0.7)]),
    '0.7 < x < 0.8': len(a[(a[0]>=0.7) & (a[0]<0.8)]),
    '0.8 < x < 0.9': len(a[(a[0]>=0.8) & (a[0]<0.9)]),
    '0.9 < x < 1': len(a[(a[0]>=0.9) & (a[0]<1)]),
    '> 1': len(a[(a[0]>=1)])
}

In [ ]:
b = pd.DataFrame(JSD_N)
b = b.fillna(0)
group_values_NS = {
    '<0.6': len(b[b[0]<0.6]),
    '0.6 < x < 0.7': len(b[(b[0]>=0.6) & (b[0]<0.7)]),
    '0.7 < x < 0.8': len(b[(b[0]>=0.7) & (b[0]<0.8)]),
    '0.8 < x < 0.9': len(b[(b[0]>=0.8) & (b[0]<0.9)]),
    '0.9 < x < 1': len(b[(b[0]>=0.9) & (b[0]<1)]),
    '> 1': len(b[(b[0]>=1)])
}

In [ ]:
data = pd.DataFrame({ 'Comentários não sexistas': JSD_N, 'Comentários sexistas': JSD_S})

In [ ]:
axes = data.hist(sharey=True,
                 sharex=True,
                 bins=25,
                 grid=True,
                 figsize=(20,8),
                 color=char_colors[1],
                 zorder=2,
                 ylabelsize=14,
                 xlabelsize=14,
                 rwidth=0.9)
plt.suptitle("Distância de Jensen Chanel dos comentário sem rótulo", fontsize=14)
plt.savefig('./images/jsd_distances.png',
         orientation='landscape',
         format='png',
         bbox_inches='tight')

## Votes

In [ ]:
try:
    votes_per_user = pd.read_csv('data/votes-per-user.csv')
except:
    print('Retrieving from original database...')
    votes_per_user = pd.read_sql_query('''
    with q as (
        select v.vote_id,
           c.comment_id,
           v.user_id,
           v.gender,
           date_part('year',age(v.birth)) as age,
           c.content,
           v.vote,
           v.avg as label
    from comments c
             join (select v1.comment_id,
                          r.avg,
                          v1.vote_id,
                          v1.serial_id as user_id,
                          v1.user_id as id,
                          v1.vote,
                          v1.gender,
                          v1.birth
                   from (
                        select u.serial_id,v2.vote,v2.vote_id, u.gender,v2.comment_id, u.birth, v2.user_id
                        from votes v2 join ( select ROW_NUMBER() OVER(ORDER BY (SELECT 1)) AS serial_id,* from users) u on v2.user_id = u.username
                       ) v1 join results r on v1.comment_id = r.comment_id) v on c.comment_id = v.comment_id
    )
    select * from q;
    ''', conn)
    votes_per_user.to_csv('data/votes-per-user.csv', index=False)

In [ ]:
votes_per_user.head(5)

In [ ]:
a = votes_per_user.groupby(by='comment_id').count()
a.groupby(by='vote_id').count()['vote']

In [ ]:
votes_per_user.groupby(by='gender').count()['user_id']

In [ ]:
x = pd.read_sql_query('select count(qtd), qtd from (select count(vote_id) as qtd from votes group by user_id) as u group by qtd;', conn)

In [ ]:
x = x.loc[:, ~x.columns.str.contains('^Unnamed')]

In [ ]:
x = x.set_index('qtd')

In [ ]:
x =x.sort_index()

In [ ]:
b = x.index.values

In [ ]:
c = x['count'].values

In [ ]:
np.mean(list(x['qtd']))

In [ ]:
np.min(list(x['qtd']))

In [ ]:
np.max(list(x['qtd']))

In [ ]:
x[x['qtd'] == 1]

In [ ]:
objects = c
y_pos = np.arange(len(objects))
performance = b 

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Usage')
plt.title('Programming language usage')

plt.show()

**Confusion Matrix to Manual Classifications**

In [ ]:
from sklearn.metrics import confusion_matrix

def matriz_confusao(gender,df):
  df = df[(votes_per_user['gender']==gender) & (votes_per_user['label']!=0.5)]
  y_true = [float(x) for x in list(df['label'].apply(define_class))]
  y_pred = [float(x) for x in list(df['vote'].apply(define_class))]
  return pd.DataFrame(confusion_matrix(y_true, y_pred, labels=[1,0]), columns=['Sr','Nr'], index=['Sv','Nv'])

female_confussion_matrix = matriz_confusao('fem',votes_per_user)
male_confussion_matrix = matriz_confusao('masc',votes_per_user)

In [ ]:
female_confussion_matrix

In [ ]:
male_confussion_matrix

In [ ]:
def precision_recal_f1(matrix):
  tp = matrix['Sr']['Sv']
  tp_fn = tp + matrix['Sr']['Nv']
  tp_fp = tp + matrix['Nr']['Sv']
  precision = tp/tp_fp
  recall = tp/tp_fn
  f1 = 2 * (precision * recall) / (precision + recall)
  return precision, recall, f1

precision_fem, recall_fem, f1_fem = precision_recal_f1(female_confussion_matrix)
precision_male, recall_male, f1_male = precision_recal_f1(male_confussion_matrix)

In [ ]:
print(f'Female metrics: {precision_fem, recall_fem, f1_fem}')
print(f'Male metrics: {precision_male, recall_male, f1_male}')

**Calculating vote info by gender/age**

In [ ]:
age_list = list(votes_per_user.age.unique())
age_list.sort()

qtd_votes_per_age = dict({})

qtd_female_votes_per_age = dict({})
qtd_sexist_female_votes_per_age = dict({})
qtd_not_sexist_female_votes_per_age = dict({})

qtd_correct_sexist_female_votes_per_age = dict({})
qtd_correct_not_sexist_female_votes_per_age = dict({})
qtd_incorrect_sexist_female_votes_per_age = dict({})
qtd_incorrect_not_sexist_female_votes_per_age = dict({})

qtd_male_votes_per_age = dict({})
qtd_sexist_male_votes_per_age = dict({})
qtd_not_sexist_male_votes_per_age = dict({})

qtd_correct_sexist_male_votes_per_age = dict({})
qtd_correct_not_sexist_male_votes_per_age = dict({})
qtd_incorrect_sexist_male_votes_per_age = dict({})
qtd_incorrect_not_sexist_male_votes_per_age = dict({})

for age in age_list:
  qtd_votes_per_age[age] = votes_per_user[votes_per_user['age']==age].shape[0]
for age in age_list:
  qtd_female_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='fem')].shape[0]
for age in age_list:
  qtd_male_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='masc')].shape[0]

for age in age_list:
  qtd_sexist_female_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='fem') & (votes_per_user['vote']==1)].shape[0]
for age in age_list:
  qtd_not_sexist_female_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='fem') & (votes_per_user['vote']==0)].shape[0]

for age in age_list:
  qtd_sexist_male_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='masc') & (votes_per_user['vote']==1)].shape[0]
for age in age_list:
  qtd_not_sexist_male_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='masc') & (votes_per_user['vote']==0)].shape[0]

for age in age_list:
  qtd_correct_sexist_female_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='fem') & (votes_per_user['vote']==1) & (votes_per_user['label']>0.5)].shape[0]
for age in age_list:
  qtd_correct_not_sexist_female_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='fem') & (votes_per_user['vote']==0) & (votes_per_user['label']<0.5)].shape[0]
for age in age_list:
  qtd_incorrect_sexist_female_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='fem') & (votes_per_user['vote']==0) & (votes_per_user['label']>0.5)].shape[0]
for age in age_list:
  qtd_incorrect_not_sexist_female_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='fem') & (votes_per_user['vote']==1) & (votes_per_user['label']<0.5)].shape[0]

for age in age_list:
  qtd_correct_sexist_male_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='masc') & (votes_per_user['vote']==1) & (votes_per_user['label']>0.5)].shape[0]
for age in age_list:
  qtd_correct_not_sexist_male_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='masc') & (votes_per_user['vote']==0) & (votes_per_user['label']<0.5)].shape[0]
for age in age_list:
  qtd_incorrect_sexist_male_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='masc') & (votes_per_user['vote']==0) & (votes_per_user['label']>0.5)].shape[0]
for age in age_list:
  qtd_incorrect_not_sexist_male_votes_per_age[age] = votes_per_user[(votes_per_user['age']==age) & (votes_per_user['gender']=='masc') & (votes_per_user['vote']==1) & (votes_per_user['label']<0.5)].shape[0]

## Charts

In [ ]:
def plot_pizza(title, labels, values):
  fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(aspect="equal"))
  recipe = labels
  data = values
  def func(pct, allvals):
      absolute = int(pct/100.*np.sum(allvals))
      return "{:.1f}%\n({:d})".format(pct, absolute)

  wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data),
                                    textprops=dict(color="w"), colors=char_colors)

  lgd = ax.legend(wedges, recipe,
            title="Classes",
            loc="center left",
            bbox_to_anchor=(1, 0, 0.5, 1))
  plt.tight_layout()
  plt.title(title,**csfont)
  plt.setp(autotexts, size=15, weight="bold")
  plt.savefig(f'./images/{stringcase.snakecase(title)}.png',
             orientation='landscape',
             format='png',
             bbox_extra_artists=(lgd,),
             bbox_inches='tight')

In [ ]:
plot_pizza('Comentários sexistas (Soma)',
           ['Likes','Dislikes'],
           [temp_comments.get('sexist').get('likes_qty'),
            temp_comments.get('sexist').get('dislikes_qty')],
          )
plot_pizza('Comentários não sexistas (Soma)',
           ['Likes','Dislikes'],
           [temp_comments.get('not_sexist').get('likes_qty'),
            temp_comments.get('not_sexist').get('dislikes_qty')]
          )
plot_pizza('Comentários não rotulados (Soma)',
           ['Likes','Dislikes'],
           [temp_comments.get('undefined').get('likes_qty'),
            temp_comments.get('undefined').get('dislikes_qty')]
          )
plot_pizza('Comentários sexistas (Mediana)',
           ['Likes','Dislikes'],
           [temp_comments.get('sexist').get('likes_qty_median'),
            temp_comments.get('sexist').get('dislikes_qty_median')]
          )
plot_pizza('Comentários não sexistas (Mediana)',
           ['Likes','Dislikes'],
           [temp_comments.get('not_sexist').get('likes_qty_median'),
            temp_comments.get('not_sexist').get('dislikes_qty_median')]

          )
plot_pizza('Comentários não rotulados (Mediana)',
           ['Likes','Dislikes'],
           [temp_comments.get('undefined').get('likes_qty_median'),
            temp_comments.get('undefined').get('dislikes_qty_median')]
          )

In [ ]:
def doublebar_plot(df_1, df_2, ages, title):
  X = np.array(ages)
  Y1 = np.array(list((df_1).values()))
  Y2 = np.array(list((df_2).values()))

  fig, ax = plt.subplots(figsize=(20,8))

  ax.axhline(0, color='black', lw=1)

  ax.bar(X, +Y1, facecolor=char_colors[0], edgecolor='white', label='Homens', width=1)
  ax.bar(X, -Y2, facecolor=char_colors[1], edgecolor='white', label='Mulheres', width=1)

  lgd = ax.legend(fontsize = 13)

  for x,y in zip(X,Y1):
      ax.text(x, y, '%d%%' % y, ha='center', va= 'bottom', fontsize = 14)
  for x,y in zip(X,Y2):
      ax.text(x, -y-0.01, '%d%%' % y, ha='center', va= 'top', fontsize = 14)

  plt.xlabel("Idades")
  plt.ylabel("Porcentagem de votos atribuídos")
  plt.xticks(X)
  plt.savefig(f'./images/{stringcase.snakecase(title)}.png',
             orientation='landscape',
             format='png',
             bbox_extra_artists=(lgd,),
             bbox_inches='tight')
  plt.show()

In [ ]:
# corretos-sexista-genero-idade
doublebar_plot(normalize_dict(qtd_correct_sexist_male_votes_per_age),
               normalize_dict(qtd_correct_sexist_female_votes_per_age),
               age_list,
               title='corretos-sexista-genero-idade'
              )

In [ ]:
# incorretos-sexista-genero-idade
doublebar_plot(normalize_dict(qtd_incorrect_sexist_male_votes_per_age),
               normalize_dict(qtd_incorrect_sexist_female_votes_per_age),
               age_list,
               title='incorretos-sexista-genero-idade'
              )

In [ ]:
# incorretos-nao-sexista-genero-idade
doublebar_plot(normalize_dict(qtd_incorrect_not_sexist_male_votes_per_age),
               normalize_dict(qtd_incorrect_not_sexist_female_votes_per_age),
               age_list,
               title='incorretos-nao-sexista-genero-idade'
              )

In [ ]:
# corretos-nao-sexista-genero-idade
doublebar_plot(normalize_dict(qtd_correct_not_sexist_male_votes_per_age),
               normalize_dict(qtd_correct_not_sexist_female_votes_per_age),
               age_list,
               title='corretos-nao-sexista-genero-idade'
              )

## Classificação Automática

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_relevant_sexist_words = relevant_sexist_words

sexist_vectorizer = TfidfVectorizer(
    stop_words=[],
    use_idf=False,
    norm=None,
    decode_error='replace',
    max_features=100,
)
not_sexist_vectorizer = TfidfVectorizer(
    stop_words=[],
    use_idf=False,
    decode_error='replace',
    max_features=100,
)
undefined_vectorizer = TfidfVectorizer(
    stop_words=[],
    use_idf=False,
    decode_error='replace',
    max_features=100,
)

sexist_doc = sexist_comments['content'].apply(lambda y: ' '.join([x for x in y.split() if x in tf_relevant_sexist_words]))
not_sexist_doc = not_sexist_comments['content'].apply(lambda y: ' '.join([x for x in y.split() if x in tf_relevant_sexist_words]))
undefined_doc = undefined_comments['content'].apply(lambda y: ' '.join([x for x in y.split() if x in tf_relevant_sexist_words]))

sexist_tf = pd.DataFrame(sexist_vectorizer.fit_transform(sexist_doc).toarray())
not_sexist_tf = pd.DataFrame(not_sexist_vectorizer.fit_transform(not_sexist_doc).toarray())
undefined_tf = pd.DataFrame(undefined_vectorizer.fit_transform(undefined_doc).toarray())

likes_df = np.array(pd.concat([sexist_comments['likes'], not_sexist_comments['likes']]).fillna(0))
dislikes_df = np.array(pd.concat([sexist_comments['dislikes'], not_sexist_comments['dislikes']]).fillna(0))
char_qty_df = np.array(pd.concat([sexist_comments['char-qty'], not_sexist_comments['char-qty']]).fillna(0))
word_qty_df = np.array(pd.concat([sexist_comments['word-qty'], not_sexist_comments['word-qty']]).fillna(0))

sexist_y = sexist_comments['avg'].apply(lambda x: 1)
not_sexist_y = not_sexist_comments['avg'].apply(lambda x: 0)
undefined_y = undefined_comments['avg'].apply(lambda x: -1)

y_df = np.array(pd.concat([sexist_y, not_sexist_y]))

tf_dataframe = pd.concat([sexist_tf_unigrams, not_sexist_tf]).fillna(0)
tf_dataframe.to_csv('./data/tf_dataframe.csv', index=False)

### **Grid Search**

In [ ]:
import pickle
from sklearn import svm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

In [ ]:
def select_features(dataframe, sub):
    X_df = dataframe[dataframe.columns[:dataframe.shape[1] -sub -1]]
    M = np.concatenate([X_df],axis=1)
    X = pd.DataFrame(M)
    return X

In [ ]:
def grid_search(file_path, method, params, method_name, X_, y, reload=False):
    tm = time.time()

    if reload:
        print('Executing Grid Search to %s.' % method_name)
        model = GridSearchCV(method, param_grid=params, cv=cv, n_jobs=8)
        model.fit(X_, y)
        pickle.dump(model, open(file_path, 'wb'))
    else:
        try:
            print('Reading %s. Model' % method_name)
            model = pickle.load(open(file_path, 'rb'))
        except:
            print('Executing Grid Search to %s.' % method_name)
            model = GridSearchCV(method, param_grid=params, cv=cv, n_jobs=8)
            model.fit(X_, y)
            pickle.dump(model, open(file_path, 'wb'))

    print('Model loaded in: @ %d seconds' %
                     ( time.time() - tm))
    print("The best parameters are %s with a score of %0.2f" % (model.best_params_, model.best_score_))
    print('-------------------------------------------------------------------------------------------')
    return model

In [ ]:
try: 
    print('...')
    dataframe = pd.read_csv('./data/dataframe.csv')
except:
    print('---')
    dataframe = tf_dataframe
    dataframe['likes'] = likes_df
    dataframe['dislikes'] = dislikes_df
    dataframe['char-qty'] = char_qty_df
    dataframe['word-qty'] = word_qty_df
    dataframe['sexist'] = y_df
    dataframe = dataframe.fillna(0)
    dataframe = tf_dataframe.sample(frac=1)
    dataframe.to_csv('.data/dataframe.csv', index=False)

In [ ]:
y_df = dataframe['sexist']
y = y_df.astype(int)
X = select_features(dataframe, 0)

In [ ]:
select_features(dataframe, 2)

In [ ]:
X_tf = select_features(dataframe, 4)
X_tf_ld = select_features(dataframe, 2)
X_tf_ld_wc = select_features(dataframe, 0)

**Support Vector Machine**

In [ ]:
cv

In [ ]:
# svm_params = dict(gamma=[0.1,1,10], C=[0.1, 1,10])
svm_params = dict(gamma=np.logspace(-9, 3, 5), C=np.logspace(-2, 10, 5))

In [ ]:
svm_tf = grid_search(
    './data/svm-tf-grid-search-model',
    SVC(),
    svm_params,
    'SVM with TF',
    X_tf_unigrams, y,
    # reload=True
)

In [ ]:
svm_tf_ld = grid_search(
    './data/svm-tf-ld-grid-search-model',
    SVC(),
    svm_params,
    'SVM with TF, likes and dislikes',
    X_tf_ld, y,
)

In [ ]:
svm_tf_ld_wc = grid_search(
    './data/svm-tf-ld-wc-grid-search-model',
    SVC(),
    svm_params,
    'SVM with TF, likes, dislikes and words&chars quantity',
    X_tf_ld_wc, y,            
)

**K-Nearest Neighbors**

In [ ]:
knn_params = dict(n_neighbors=[3, 5, 11, 19], weights=['uniform', 'distance'], metric=['euclidean', 'manhattan'])

In [ ]:
knn_tf = grid_search(
    './data/knn-tf-grid-search-model',
    KNeighborsClassifier(),
    knn_params,
    'KNN with TF',
    X_tf_unigrams, y,
    # reload=True
)

In [ ]:
knn_tf_ld = grid_search(
    './data/knn-tf-ld-grid-search-model',
    KNeighborsClassifier(),
    knn_params,
    'KNN with TF, likes and dislikes',
    X_tf_ld, y,
    # reload=True
)


In [ ]:
knn_tf_ld_wc = grid_search(
    './data/knn-tf-ld-wc-grid-search-model',
    KNeighborsClassifier(),
    knn_params,
    'KNN with TF, likes, dislikes words and chars qauntitys',
    X_tf_ld_wc, y,
    reload=True
)

**Classification reports to single test**

In [ ]:
from sklearn.model_selection import train_test_split
# Only TF
X1_train, X1_test, y1_train, y1_test = train_test_split(X_tf_unigrams, y, stratify=y)
# TF, Likes and Dislikes
X2_train, X2_test, y2_train, y2_test = train_test_split(X_tf_ld, y, stratify=y)
# TF, Likes, Dislikes, Words and Chars quantities
X3_train, X3_test, y3_train, y3_test = train_test_split(X_tf_ld_wc, y, stratify=y)

In [ ]:
from sklearn.metrics import classification_report

def report_results(X_test, y_test, model):
    y_ = model.predict(X_test)
    report = classification_report( y_test, y_ )
    c_matrix = confusion_matrix(y_test, y_,  labels=[1,0])
    return report, c_matrix

In [ ]:
svm_tf_report, svm_tf_cmatrix = report_results(X1_test, y1_test, svm_tf)
svm_tf_ld_report, svm_tf_ld_cmatrix = report_results(X2_test, y2_test, svm_tf_ld)
svm_tf_ld_wc_report, svm_tf_ld_wc_cmatrix = report_results(X3_test, y3_test, svm_tf_ld_wc)
knn_tf_report, knn_tf_cmatrix = report_results(X1_test, y1_test, knn_tf)
knn_tf_ld_report, knn_tf_ld_cmatrix = report_results(X2_test, y2_test, knn_tf_ld)
knn_tf_ld_wc_report, knn_tf_ld_wc_cmatrix = report_results(X3_test, y3_test, knn_tf_ld_wc)

In [ ]:
print('>>> SVM with TF results')
print(svm_tf_report)
print('Confusion matrix')
print(pd.DataFrame(svm_tf_cmatrix, columns=['T','F'], index=['F','T']))
print('>>> SVM with TF, Likes and Dislikes results')
print(svm_tf_ld_report)
print('Confusion matrix')
print(pd.DataFrame(svm_tf_ld_cmatrix, columns=['T','F'], index=['F','T']))
print('>>> SVM with TF, Likes, Dislikes, Words and Chars results')
print(svm_tf_ld_wc_report)
print('Confusion matrix')
print(pd.DataFrame(svm_tf_ld_wc_cmatrix, columns=['T','F'], index=['F','T']))
print('>>> KNN with TF results')
print(knn_tf_report)
print('Confusion matrix')
print(pd.DataFrame(knn_tf_cmatrix, columns=['T','F'], index=['F','T']))
print('>>> KNN with TF, Words e Chars quantities results')
print(knn_tf_ld_report)
print('Confusion matrix')
print(pd.DataFrame(knn_tf_ld_cmatrix, columns=['T','F'], index=['F','T']))
print('>>> KNN with TF, Likes, Dislikes, Words e Chars quantities results')
print(knn_tf_ld_wc_report)
print('Confusion matrix')
print(pd.DataFrame(knn_tf_ld_wc_cmatrix, columns=['T','F'], index=['F','T']))

## Not labeled classification

In [ ]:
y_df = np.array(pd.concat([undefined_y]))
tf_dataframe = pd.concat([undefined_tf]).fillna(0)
dataframe = tf_dataframe
dataframe['sexist'] = y_df
dataframe = dataframe.fillna(0)
dataframe = tf_dataframe.sample(frac=1)

y_df = dataframe['sexist']
y = y_df.astype(int)
X = select_features(dataframe, 0)

y_ = svm_tf.predict(X)
unique, counts = np.unique(y_, return_counts=True)
dict(zip(unique, counts))

In [ ]:
pd.DataFrame(y_)

In [ ]:
undefined_comments = pd.concat([undefined_comments, pd.DataFrame(y_)])
undefined_comments

In [ ]:
tf_dataframe